In [14]:
# ==============================================================================
# 1. 필수 라이브러리 설치 및 임포트
# ==============================================================================
# !pip install -q groq pandas numpy scikit-learn requests

import json
import time
import re
import requests
import getpass
import pandas as pd
import numpy as np
from groq import Groq
from sklearn.metrics import recall_score, precision_score, f1_score
from google.colab import userdata
from IPython.display import clear_output

# ==============================================================================
# 2. 데이터 로드 (GitHub Raw URL 기준)
# ==============================================================================
DATA_URL = "https://raw.githubusercontent.com/heeumm/heeummm/main/data.json"

try:
    response = requests.get(DATA_URL)
    response.raise_for_status()
    data = response.json()
    print(f"데이터 로드 성공: {DATA_URL}")
    print(f"- 분석 대상 유저 이력: {len(data['User_History'])}건")
    print(f"- 분석 대상 추천 목록: {len(data['Recommendation_List'])}건")
except Exception as e:
    print(f"데이터 로드 실패: {e}")
    print("💡 URL이 정확한지, 혹은 저장소가 Public인지 확인해 주세요.")

# ==============================================================================
# 3. API 세팅 및 에이전트 정의 (보안 강화 버전)
# ==============================================================================
try:
    # 1순위: Colab Secrets(userdata) 확인
    MY_API_KEY = userdata.get('GROQ_API_KEY')
    if MY_API_KEY is None: raise Exception
    print("환경 변수에서 API Key를 로드했습니다.")
except:
    # 2순위: userdata에 없을 경우 보안 입력창 활성화
    MY_API_KEY = getpass.getpass("🔑 Groq API Key를 입력하세요 (화면에 보이지 않습니다): ")
    clear_output(wait=True)
    print(" API Key 입력 완료 (보안을 위해 출력창을 초기화).")

client = Groq(api_key=MY_API_KEY)
MODEL_NAME = "llama-3.1-8b-instant"

def calculate_distance(v1, v2):
    r"""유클리드 거리 측정 ($$d(p, q) = \sqrt{\sum_{i=1}^{n} (p_i - q_i)^2}$$)"""
    return np.linalg.norm(np.array(v1) - np.array(v2))

def get_baseline_vector(item):
    """[Baseline: 1D Genre Bot]"""
    prompt = f"Rate '{item['title']}' for 'Noir/Crime' genre (1-5). Output ONLY the integer."
    try:
        res = client.chat.completions.create(model=MODEL_NAME, messages=[{"role": "user", "content": prompt}], temperature=0.1)
        score = re.search(r'[1-5]', res.choices[0].message.content)
        return np.array([int(score.group())]) if score else np.array([3])
    except: return np.array([3])

def get_agentic_vector(item):
    """[Proposed: 4D DNA Scan]"""
    prompt = f"""[Agentic Audit] Analyze '{item['title']}' for:
    1. Genre, 2. Subject, 3. Tone, 4. Complexity.
    Rate each 1-5. Output ONLY a JSON list of 4 integers. Example: [5, 4, 3, 5]"""
    try:
        res = client.chat.completions.create(model=MODEL_NAME, messages=[{"role": "user", "content": prompt}], temperature=0.1)
        nums = re.findall(r'[1-5]', res.choices[0].message.content)
        return np.array([int(n) for n in nums[:4]]) if len(nums) >= 4 else np.array([3, 3, 3, 3])
    except: return np.array([3, 3, 3, 3])

# ==============================================================================
# 4. 필터 버블 심층 검증 (Distance Measurement)
# ==============================================================================
print(f"\n {MODEL_NAME} 기반 필터 버블 심층 검증 시작")

v_history_avg = np.mean([get_baseline_vector(item) for item in data['User_History']], axis=0)
h_history_avg = np.mean([get_agentic_vector(item) for item in data['User_History']], axis=0)

results = []
for item in data['Recommendation_List']:
    v_dist = calculate_distance(v_history_avg, get_baseline_vector(item))
    p_dist = calculate_distance(h_history_avg, get_agentic_vector(item))
    results.append({
        "Title": item['title'], "Baseline_Dist": round(float(v_dist), 4),
        "Proposed_Dist": round(float(p_dist), 4)
    })
    time.sleep(0.3)

df = pd.DataFrame(results)

# ==============================================================================
# 5. 성능 지표 산출 (Bottom 20% Threshold)
# ==============================================================================
B_THRESHOLD = np.percentile(df['Baseline_Dist'], 20)
P_THRESHOLD = np.percentile(df['Proposed_Dist'], 20)

y_true = [1]*5 + [0]*20
y_pred_b = [1 if x <= B_THRESHOLD else 0 for x in df['Baseline_Dist']]
y_pred_p = [1 if x <= P_THRESHOLD else 0 for x in df['Proposed_Dist']]

metrics = {
    'Metric': ['Precision', 'Recall', 'F1-score'],
    'Baseline (1D)': [
        round(precision_score(y_true, y_pred_b, zero_division=0), 4),
        round(recall_score(y_true, y_pred_b, zero_division=0), 4),
        round(f1_score(y_true, y_pred_b, zero_division=0), 4)
    ],
    'Proposed (4D)': [
        round(precision_score(y_true, y_pred_p, zero_division=0), 4),
        round(recall_score(y_true, y_pred_p, zero_division=0), 4),
        round(f1_score(y_true, y_pred_p, zero_division=0), 4)
    ]
}

# ==============================================================================
# 6. 최종 리포트 출력
# ==============================================================================
print("\n" + "="*80)
print(f" 실험 결과 요약 (Euclidean Distance 기반)")
print("-" * 80)
print(pd.DataFrame(metrics).to_string(index=False))
print("="*80)

 API Key 입력 완료 (보안을 위해 출력창을 초기화).

 llama-3.1-8b-instant 기반 필터 버블 심층 검증 시작

 실험 결과 요약 (Euclidean Distance 기반)
--------------------------------------------------------------------------------
   Metric  Baseline (1D)  Proposed (4D)
Precision         0.1429         0.2857
   Recall         0.4000         0.4000
 F1-score         0.2105         0.3333
